In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model
import tensorflow as tf

In [ ]:
# specify the directories for the train, test, and validation datasets
train_dir = '/content/drive/MyDrive/Cancer Detection Project/train'
test_dir = '/content/drive/MyDrive/Cancer Detection Project/test'
val_dir = '/content/drive/MyDrive/Cancer Detection Project/valid'


In [ ]:
# create a data generator for the train, test, and validation datasets
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# prepare the generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 1460 images belonging to 6 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 477 images belonging to 6 classes.


In [ ]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 142 images belonging to 6 classes.


In [ ]:
# extract features using VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

58889256/58889256 [==============================] - 1s 0us/step


In [9]:
train_features = vgg.predict_generator(train_generator, steps=len(train_generator))


<ipython-input-9-92ddde6857a8>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  train_features = vgg.predict_generator(train_generator, steps=len(train_generator))


In [ ]:
test_features = vgg.predict_generator(test_generator, steps=len(test_generator))

<ipython-input-10-e613130ff8f7>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  test_features = vgg.predict_generator(test_generator, steps=len(test_generator))


In [ ]:
val_features = vgg.predict_generator(val_generator, steps=len(val_generator))

<ipython-input-11-2c058fbae966>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  val_features = vgg.predict_generator(val_generator, steps=len(val_generator))


In [ ]:
# flatten the features
train_features = np.reshape(train_features, (train_features.shape[0], 7*7*512))
test_features = np.reshape(test_features, (test_features.shape[0], 7*7*512))
val_features = np.reshape(val_features, (val_features.shape[0], 7*7*512))

In [ ]:
# encode the labels
le = LabelEncoder()
train_labels = le.fit_transform(train_generator.classes)
test_labels = le.transform(test_generator.classes)
val_labels = le.transform(val_generator.classes)

In [ ]:
from keras.utils import to_categorical

# one-hot encode the labels
train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# create the model
input_tensor = Input(shape=(7*7*512,))
x = Dense(1024, activation='relu')

In [ ]:
x = Dense(1024, activation='relu')(input_tensor)
x = Dense(512, activation='relu')(x)
output_tensor = Dense(6, activation='softmax')(x)


In [ ]:
model = Model(input_tensor, output_tensor)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# train the model
history = model.fit(train_features, train_labels, epochs=50, validation_data=(val_features, val_labels))

Epoch 1/50
46/46 [==============================] - 6s 119ms/step - loss: 3.7232 - accuracy: 0.2507 - val_loss: 1.8223 - val_accuracy: 0.2606
Epoch 2/50
46/46 [==============================] - 6s 125ms/step - loss: 1.6097 - accuracy: 0.3349 - val_loss: 1.8795 - val_accuracy: 0.1408
Epoch 3/50
46/46 [==============================] - 6s 125ms/step - loss: 1.5695 - accuracy: 0.3534 - val_loss: 1.7686 - val_accuracy: 0.3803
Epoch 4/50
46/46 [==============================] - 5s 114ms/step - loss: 1.4939 - accuracy: 0.4014 - val_loss: 1.7733 - val_accuracy: 0.3732
Epoch 5/50
46/46 [==============================] - 6s 136ms/step - loss: 1.4635 - accuracy: 0.4212 - val_loss: 1.9026 - val_accuracy: 0.1831
Epoch 6/50
46/46 [==============================] - 5s 115ms/step - loss: 1.3948 - accuracy: 0.4575 - val_loss: 1.9277 - val_accuracy: 0.3732
Epoch 7/50
46/46 [==============================] - 6s 137ms/step - loss: 1.3413 - accuracy: 0.4890 - val_loss: 2.0566 - val_accuracy: 0.1972
Epoch 

In [ ]:
# evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_features, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
# predict the labels for the test set
predictions = model.predict(test_features)


In [ ]:
# print the classification report
print(classification_report(test_labels, predictions.argmax(axis=1)))
